In [ ]:
import geopandas as gpd
from ipyfilechooser import FileChooser
import numpy as np
from pathlib import Path
import rasterio as rio
import rioxarray as rxr
import rioxarray.merge
from shapely.geometry import mapping

**This notebook downloads Pekel Occurrence data intersecting the chip, clips each Pekel scene to the extents of the chip, and then merges them**

### Select the chip directory

In [ ]:
print("Select the chip directory containing the Planet data, which was generated in Notebook 0")
fc = FileChooser(Path.cwd())
display(fc)

In [ ]:
chip_dir = Path(fc.selected_path)
chip_id = chip_dir.name

shape_path = list(chip_dir.rglob('*.shp'))
if len(shape_path) < 1:
    raise FileNotFoundError
elif len(shape_path) > 1:
    raise Exception("Multiple shapefiles found")
else:
    shape_path = shape_path[0]

planet_path = list(chip_dir.rglob('*_AnalyticMS_*.tif'))
if len(planet_path) < 1:
    raise FileNotFoundError
elif len(planet_path) > 1:
    raise Exception("Multiple Planet data files found")
else:
    planet_path = planet_path[0]
    
print(shape_path)
print(planet_path)

### Load the chip geometry from the chip shapefile

In [ ]:
geometry_gdf = gpd.read_file(shape_path)
geometry = geometry_gdf['geometry'][0]
geometry

### Find the Planet Data UTM

In [ ]:
with rio.open(planet_path) as planet:
    print(planet.crs)
    print(planet.transform)
    planet_crs = planet.crs
    planet_transform = planet.transform

### Load Pekel Tile geojson into a dataframe

In [ ]:
peckel_tile_gdf = gpd.read_file('peckel_tiles.geojson')
peckel_tile_gdf.head()

### Create a dataframe of tiles intersecting the chip

In [ ]:
intersections = peckel_tile_gdf.geometry.intersects(geometry)
peckel_subset = peckel_tile_gdf[intersections].reset_index(drop=True)
peckel_subset

### Download tiles and clip them to chip geometry

In [ ]:
to_merge = list()
for url in peckel_subset.source_url:
    
    # pekel_tiles.goejson contains broken urls for longitudes < 100W or 100E
    # example: '060W' must be converted to '60W'
    fname_split = url.split('/')[-1].split('_')
    if fname_split[1][0] == '0':
        fname_split[1] = fname_split[1][1:]
        fname = '_'.join(fname_split)
        url = f"{'/'.join(url.split('/')[:-1])}/{fname}"

    tile = rxr.open_rasterio(url, masked=True).squeeze()
    
    # tile.rio.to_raster(chip_dir/f"{pekel_name}_{chip_id}.tif")
#     tile = tile.rio.clip(geometry_gdf.geometry.apply(mapping), 'EPSG:4326')
#     to_merge.append(tile)
# to_merge

In [ ]:
print(peckel_subset.source_url[0])

### Merge clipped tiles and output to GeoTiff

In [ ]:
pekel_name = Path(url.split('/')[-1]).stem

merged = rxr.merge.merge_arrays(to_merge)
merged.rio.to_raster(chip_dir/f"{pekel_name}_{chip_id}.tif")
merged